### N Way Conversation - Coffee Talk 

This example simulates an N-way conversation between the characters of the Saturday Night Live skit Coffee Talk.

The character information is retrieved from a model and each character is handled by its own model selected at random from a list of available models. Only the number of characters, number of rounds, and available models are configured.

The example can use OpenRouter, OpenAI, or Ollama, in that order. 

In [ ]:
# Setup ...

# The number of characters (models) conversing
NBR_CHARACTERS=4

# The number of rounds of conversation
NBR_ROUNDS=4

# Available OpenRouter models. The base model is used to select characters and the topic. Other models are used for the conversation
OPENROUTER_MODELS="openai/gpt-4.1-mini, anthropic/claude-3.5-haiku, google/gemini-2.5-flash"
OPENROUTER_BASE="openai/gpt-5"

# Available OpenAI models
OPENAI_MODELS="gpt-4.1, gpt-4.1-mini, gpt-5-nano"
OPENAI_BASE="gpt-5"

# Available Ollama models. Note that these must be pre-fetched or errors will occur (and won't be handled)
OLLAMA_MODELS="gpt-oss, gemma3, llama3.2"
OLLAMA_BASE="gpt-oss"


In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Setup the LLM client and models. OpenRouter has priority if available, then OpenAI, then Ollama.

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}, using OpenRouter.")
    available_models=OPENROUTER_MODELS
    base_model=OPENROUTER_BASE
    client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouter_api_key)
elif openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}, using OpenAI.")
    available_models=OPENAI_MODELS
    base_model=OPENAI_BASE
    client = OpenAI()
else:
    print("OpenAI API Key not set, using Ollama.")
    available_models=OLLAMA_MODELS
    base_model=OLLAMA_BASE
    client = OpenAI(api_key="ollama", base_url="http://localhost:11434/v1")

In [ ]:
# Get the characters from the base model
system_prompt = """
You will be asked to return information about characters in the SNL skit Coffee Talk
You should return the information as a JSON response with the following format:
{
  { "name" : "Linda", "persona", "....", "model" : "model-name" },
  { "name" : "Paul", "persona", "....", "model" : "model-name" }
}

"""

user_prompt = f"""
Create a list of the many characters from the SNL skit Coffee Talk, and return {NBR_CHARACTERS} total characters.
Always return Linda Richmond as the first character.
Return one caller.
Select the remaining characters at random from the list of all characters. 
For the model value, return a random model name from this list: {available_models}.
"""

response = client.chat.completions.create(
        model=base_model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
result = response.choices[0].message.content
characters = json.loads(result)

print(json.dumps(characters, indent=2))


In [ ]:
# Generate system prompts for each character, which includes their name, persona, the other guests, and how they should respond.

guests = "The guests on todays show are "
guest_names = [character['name'] for character in characters["characters"]]
guests += ", ".join(guest_names)

prompt = ""
for character in characters["characters"]:
    prompt = f"You are {character['name']} a character on the SNL skit Coffee Talk."
    prompt += f" Your personality is : {character['persona']} "
    prompt += " " + guests + "."
    prompt += " Keep responses brief and in character."
    prompt += " In the conversation history, each response is prefixed with the character's name to identify the respondent."
    prompt += " Your response should not include your character name as a prefix."

    character["system_prompt"] = prompt

print(json.dumps(characters, indent=2))


In [ ]:
# Get the topic
user_prompt="""
In the SNL skit Coffee Talk, the host Linda Richmond proposes topics in the form "X Y is neither X, nor Y - discuss".
Create a list of the many topics proposed on the show, and select one at random and return it.
Return only the selected topic without any formatting.
"""

response = client.chat.completions.create(
        model=base_model,
        messages=[
            {"role": "user", "content": user_prompt}
        ],
    )
topic = response.choices[0].message.content

print(topic)

In [ ]:
def get_character_response(character,history):
    user_prompt = f"""
    The conversation so far is as follows:
    {history}
    What is your response? 
    """
    
    response = client.chat.completions.create(
        model=character["model"],
        messages=[
            {"role": "system", "content": character["system_prompt"]},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
    

In [ ]:
# Start the show!

history = ""
history += "Welcome to Coffee Talk, I am your host Linda Richmond. Today's guests are:\n"

for character in characters["characters"][1:]:
    history += f"  - {character['name']}\n"

history += f"\nI'll give you a topic: {topic}\n"

display(Markdown("---"))
display(Markdown(history))
display(Markdown("---"))

# Other guests respond (first round)
for character in characters["characters"][1:]:
    response = get_character_response(character,history)
    display(Markdown(f"**{character['name']}({character['model']}):** {response}"))  
    history += f"\n{character['name']}: {response}"

# Continue conversation for remaining rounds (all characters including Linda)
for round in range(1, NBR_ROUNDS):
    for character in characters["characters"]:
        response = get_character_response(character,history)
        display(Markdown(f"**{character['name']}({character['model']}):** {response}"))  
        history += f"\n{character['name']}: {response}"

# Wrap it up
user_prompt=f"""
It's time to wrap up the show. Here's the whole conversation:\n
{history}
Wrap up the show, as only you can.
"""

linda = characters["characters"][0]
response = client.chat.completions.create(
        model=linda["model"],
        messages=[
            {"role": "system", "content": linda["system_prompt"]},
            {"role": "user", "content": user_prompt}
        ]
    )

display(Markdown("---"))
display(Markdown(response.choices[0].message.content))  
